# Funnel analysis part 3 

1. Putting all the chunks together and calculating the funnel stages
2. Putting my data in csv to visualize in Power Bi
3. Calculating the total conversion rates for each event over 6 month period 

In [1]:
import pandas as pd


In [2]:
# Where in the funnel is the biggest drop-off? Using the 3 data chunks created previously.

# Load the data

view_chunk1 = pd.read_parquet("df_view1.parquet")
view_chunk2 = pd.read_parquet("df_view2.parquet")
view_chunk3 = pd.read_parquet("df_view3.parquet")

In [3]:
cart_chunk1 = pd.read_parquet("df_cart1.parquet")
cart_chunk2 = pd.read_parquet("df_cart2.parquet")
cart_chunk3 = pd.read_parquet("df_cart3.parquet")

In [4]:
purchase_chunk1 = pd.read_parquet("df_purchase1.parquet")
purchase_chunk2 = pd.read_parquet("df_purchase2.parquet")
purchase_chunk3 = pd.read_parquet("df_purchase3.parquet")

In [5]:
# Combine chunks for overall analysis if needed

all_views = pd.concat([view_chunk1, view_chunk2, view_chunk3])
all_carts = pd.concat([cart_chunk1, cart_chunk2, cart_chunk3])
all_purchases = pd.concat([purchase_chunk1, purchase_chunk2, purchase_chunk3])

In [6]:
all_views.head()

,event_type,user_id,event_date,event_timestamp
0,view,541312140,2019-10-01,00:00:00
1,view,554748717,2019-10-01,00:00:00
2,view,519107250,2019-10-01,00:00:01
3,view,550050854,2019-10-01,00:00:01
4,view,535871217,2019-10-01,00:00:04


In [ ]:
# Based on my visualization and finding significant anomaly on February 27th 2019 I decided to exclude data from that day in my analysis

feb_clean = feb_raw[feb_raw['date'] != datetime.date(2020, 2, 27)]

In [7]:
# Conversion rates grouped by months + unique users

views_per_month = all_views.groupby("event_date")['user_id'].nunique()
carts_per_month = all_carts.groupby('event_date')['user_id'].nunique()
purchases_per_month = all_purchases.groupby('event_date')['user_id'].nunique()

In [8]:
# Calculation of conversion rates per month 

view_to_cart_by_month = carts_per_month / views_per_month
view_to_purchase_by_month = purchases_per_month / views_per_month
cart_to_purchase_by_month = purchases_per_month / carts_per_month

In [9]:
# Convert Series to DataFrame for Power BI export

df_conversion = pd.DataFrame({
    'views_per_month': views_per_month,
    'carts_per_month': carts_per_month,
    'purchases_per_month': purchases_per_month,
    'view_to_cart': view_to_cart_by_month,
    'view_to_purchase': view_to_purchase_by_month,
    'cart_to_purchase': cart_to_purchase_by_month
})

# Export to CSV
df_conversion.to_csv('conversion_analysis.csv', index=True)

In [10]:
# Calculating conversion rates for funnel view to cart over the 6 months

ult_view_to_cart = (cart_chunk1 ["user_id"].nunique() +
                    cart_chunk2 ["user_id"].nunique() +
                    cart_chunk3 ["user_id"].nunique()) / (view_chunk1 ["user_id"].nunique() +
                                        view_chunk2 ["user_id"].nunique() + view_chunk3 ["user_id"].nunique())

In [8]:
# Calculaing conversion rates for funnel view to purchase over the 6 months

ult_view_to_purchase = (purchase_chunk1 ["user_id"].nunique() +
                    purchase_chunk2 ["user_id"].nunique() +
                    purchase_chunk3 ["user_id"].nunique()) / (view_chunk1 ["user_id"].nunique() +
                                        view_chunk2 ["user_id"].nunique() + view_chunk3 ["user_id"].nunique())

In [9]:
# Calculating conversion rates for funnel cart to purchase over the 6 months

ult_cart_to_purchase = (purchase_chunk1 ["user_id"].nunique() +
                        purchase_chunk2 ["user_id"].nunique() +
                        purchase_chunk3 ["user_id"].nunique()) / (cart_chunk1 ["user_id"].nunique() +
                                        cart_chunk2 ["user_id"].nunique() + cart_chunk3 ["user_id"].nunique())

In [22]:
print(f"View to cart conversion rate: {ult_view_to_cart:.2%}")
print(f"View to purchase conversion rate: {ult_view_to_purchase:.2%}")
print(f"Cart to purchase conversion rate: {ult_cart_to_purchase:.2%}")


View to cart conversion rate: 20.29%
View to purchase conversion rate: 11.47%
Cart to purchase conversion rate: 56.55%
